# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 12:10PM,239331,10,0085754843,ISDIN Corporation,Released
1,Jun 10 2022 12:10PM,239331,10,0085754849,ISDIN Corporation,Released
2,Jun 10 2022 12:10PM,239331,10,0085754862,ISDIN Corporation,Released
3,Jun 10 2022 12:10PM,239331,10,0085754947,ISDIN Corporation,Released
4,Jun 10 2022 12:10PM,239331,10,0085754956,ISDIN Corporation,Released
5,Jun 10 2022 12:10PM,239331,10,0085754957,ISDIN Corporation,Released
6,Jun 10 2022 12:10PM,239331,10,0085754968,ISDIN Corporation,Released
7,Jun 10 2022 11:57AM,239328,12,HH6203,Hush Hush,Released
8,Jun 10 2022 11:57AM,239328,12,HH6226,Hush Hush,Released
9,Jun 10 2022 11:56AM,239329,21,399290117,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,239326,Released,1
26,239327,Released,1
27,239328,Released,2
28,239329,Released,1
29,239331,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239326,NaN,NaN,1.0
239327,NaN,NaN,1.0
239328,NaN,NaN,2.0
239329,NaN,NaN,1.0
239331,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239253,0.0,0.0,1.0
239258,0.0,0.0,1.0
239261,0.0,1.0,1.0
239286,0.0,0.0,1.0
239291,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239253,0,0,1
239258,0,0,1
239261,0,1,1
239286,0,0,1
239291,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239253,0,0,1
1,239258,0,0,1
2,239261,0,1,1
3,239286,0,0,1
4,239291,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239253,,,1
1,239258,,,1
2,239261,,1,1
3,239286,,,1
4,239291,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation
7,Jun 10 2022 11:57AM,239328,12,Hush Hush
9,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc."
10,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc."
11,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc."
12,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc."
13,Jun 10 2022 11:05AM,239323,20,"ACI Healthcare USA, Inc."
22,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc."
23,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc."
24,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,,,7
1,Jun 10 2022 11:57AM,239328,12,Hush Hush,,,2
2,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",,,1
3,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",,,1
4,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",,,1
5,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc.",,,1
6,Jun 10 2022 11:05AM,239323,20,"ACI Healthcare USA, Inc.",,3,6
7,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",,,1
8,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",,,1
9,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7,,
1,Jun 10 2022 11:57AM,239328,12,Hush Hush,2,,
2,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",1,,
4,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",1,,
5,Jun 10 2022 11:17AM,239324,19,"Granules Pharmaceuticals, Inc.",1,,
6,Jun 10 2022 11:05AM,239323,20,"ACI Healthcare USA, Inc.",6,3,
7,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",1,,
8,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",1,,
9,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7,,
1,Jun 10 2022 11:57AM,239328,12,Hush Hush,2,,
2,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",1,,
4,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7.0,NaN,NaN
1,Jun 10 2022 11:57AM,239328,12,Hush Hush,2.0,NaN,NaN
2,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 12:10PM,239331,10,ISDIN Corporation,7.0,0.0,0.0
1,Jun 10 2022 11:57AM,239328,12,Hush Hush,2.0,0.0,0.0
2,Jun 10 2022 11:56AM,239329,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 10 2022 11:45AM,239327,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 10 2022 11:23AM,239326,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,478646,16.0,16.0,4.0
12,717895,3.0,12.0,0.0
15,717912,12.0,26.0,0.0
16,717895,21.0,0.0,0.0
18,239253,1.0,0.0,0.0
19,478640,1.0,8.0,0.0
20,478636,26.0,3.0,0.0
21,2153787,9.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,478646,16.0,16.0,4.0
1,12,717895,3.0,12.0,0.0
2,15,717912,12.0,26.0,0.0
3,16,717895,21.0,0.0,0.0
4,18,239253,1.0,0.0,0.0
5,19,478640,1.0,8.0,0.0
6,20,478636,26.0,3.0,0.0
7,21,2153787,9.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,16.0,4.0
1,12,3.0,12.0,0.0
2,15,12.0,26.0,0.0
3,16,21.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,1.0,8.0,0.0
6,20,26.0,3.0,0.0
7,21,9.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,12,Released,3.0
2,15,Released,12.0
3,16,Released,21.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,12.0,26.0,0.0,0.0,8.0,3.0,0.0
Released,16.0,3.0,12.0,21.0,1.0,1.0,26.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,12.0,26.0,0.0,0.0,8.0,3.0,0.0
2,Released,16.0,3.0,12.0,21.0,1.0,1.0,26.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,12.0,26.0,0.0,0.0,8.0,3.0,0.0
2,Released,16.0,3.0,12.0,21.0,1.0,1.0,26.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()